# Covid-19 Dashboard - Test to Case Comparisons

Public health England publish daily status' showing the number of new tests conducted under the Covid-19 Pandemic and the results. 

This dashboard allows you to visualise this data in a way that you can determine the relationship between tests conducted and the number of cases reported as a result. 

This is with a view to build a perspective on the US President's claim of more cases only being discovered due to increased testing. This was documented widely in the news including [here](https://www.cnbc.com/2020/06/23/trump-blames-rise-in-coronavirus-cases-on-testing-despite-signs-of-spread.html) on CNBC (a US news provider). 

You can refresh this data on a daily basis as and when Public Health England make it available by making use of the provided 'Refresh Button'

In [13]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [14]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [15]:
filters = [
    'areaType=overview' #data for the entire United Kingdom
]
structure = {
    "date": "date",
    "casesPub": "newCasesByPublishDate",
    "casesSpec": "newCasesBySpecimenDate",
    "testsNew": "newTestsByPublishDate",
    "testsTotalPub": "cumTestsByPublishDate", 
    "testsTotalSpec": "cumCasesBySpecimenDate"
    '''this data will allow for further analysis at a later date to compare total numbers of tests and cases'''
}
api = Cov19API(filters=filters, structure=structure) #define the api
testcasedata=api.get_json() 
with open("testcasedata.json", "wt") as OUTF:
    json.dump(testcasedata, OUTF)

In [16]:
testcasedatalist=testcasedata['data'] #create a list from the api, original was a dictionary 

testdates=[dictionary['date'] for dictionary in testcasedatalist ]
testdates.sort()# sort the dates in chronological order

In [17]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

In [18]:
startdate=parse_date(testdates[0])
enddate=parse_date(testdates[-1])

In [19]:
index=pd.date_range(startdate, enddate, freq='D')
newcasetest=pd.DataFrame(index=index, columns=['casesPub', 'casesSpec', 'testsNew']) #create the data frame

In [20]:
for entry in testcasedatalist: # populate the data frame with data from the list created
    date=parse_date(entry['date'])
    for column in ['casesPub', 'casesSpec', 'testsNew']:
        if pd.isna(newcasetest.loc[date, column]):  
            value= float(entry[column]) if entry[column]!=None else 0.0 # display null data with 0 instead of none
            newcasetest.loc[date, column]=value
            
newcasetest.fillna(0.0, inplace=True)

## Tests Against Cases

The US President had publicly stated his hypothesis that the reason that more Covid cases were arising were due to increase testing. 

Through the gathering of UK health data and the plotting of this against the metrics of cases and tests, a visual representation of the relationship between increased testing and cases found can be seen. 

From the data to date (which can be refreshed below), the US Presidents hypothesis is proven incorrect. This is very clear between May and October 2020 where testing increased by 500%, and cases in most case levelled or decreased. 

#### Key: 

  - casesPub: The number of cases recored by publish date 
  - casesSpec: The number of cases recorded by Specimen date 
  - testsNew: The number of new tests recorded

By default, for your interest all metrics are plotted. 

An interactive control is available which can be used to add or remove metrics. 

In [21]:
def access_api(api):
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    return {} # return data read from the API

In [22]:
casetestcols=wdg.SelectMultiple(
    options=['casesPub', 'casesSpec', 'testsNew'],
    value=['casesPub', 'casesSpec', 'testsNew'],
    rows=3,
    description='Stats',
    disabled=False
)

def casetestgraph(graphcolumns):
    # our callback function.
    ncols=len(graphcolumns)
    if ncols>0:
        newcasetest.plot(kind='line', y=list(graphcolumns)) 
    else:
        # if the user has not selected any column, print a message instead
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
    
   
casetestgraph=wdg.interactive_output(casetestgraph, {'graphcolumns': casetestcols})

display(casetestcols, casetestgraph)

SelectMultiple(description='Stats', index=(0, 1, 2), options=('casesPub', 'casesSpec', 'testsNew'), rows=3, va…

Output()

In [23]:
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """ 
    apidata=api
    global df
    df=access_api(newcasetest)

apibutton=wdg.Button(
    description='Fetch Data', 
    disabled=False,
    button_style='success', 
    tooltip="click button to refresh data",
    
    icon='refresh'
)


apibutton.on_click(api_button_callback) 

display(apibutton)

Button(button_style='success', description='Fetch Data', icon='refresh', style=ButtonStyle(), tooltip='click b…

Copyright for all test and case data: UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).